<a href="https://colab.research.google.com/github/Chaitya0623/SentenceRestructuring/blob/main/SentenceRestructuring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentence Restructuring with user-controlled Difficulty

#Preprocessing the Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research Papers/Sentence Restructuring/synonyms.csv')
df

,lemma,part_of_speech,synonyms
0,.22-caliber,adjective,.22 caliber;.22 calibre;.22-calibre
1,.22-calibre,adjective,.22 caliber;.22-caliber;.22 calibre
2,.22 caliber,adjective,.22-caliber;.22 calibre;.22-calibre
3,.22 calibre,adjective,.22 caliber;.22-caliber;.22-calibre
4,.38-caliber,adjective,.38 caliber;.38 calibre;.38-calibre
...,...,...,...
126996,zero in,verb,range in;home in|zero
126997,zip by,verb,fly by;whisk by
126998,zip up,verb,zipper;zip
126999,zonk out,verb,pass out;black out


In [ ]:
df = df[df['part_of_speech'] != 'noun']
df = df.reset_index(drop=True)

df['synonyms'] = df['synonyms'].str.split('[|;]')
df = df.dropna()

df['synonyms'] = df.apply(lambda row: row['synonyms'] + [row['lemma']], axis=1)
df = df.drop('lemma',axis=1)

# convert each list to a frozenset to handle unordered strings
df['synonyms'] = df['synonyms'].apply(frozenset)

# drop the duplicate frozensets
df.drop_duplicates(subset=['synonyms'], inplace=True)

# convert the frozensets back to lists
df['synonyms'] = df['synonyms'].apply(list)

df = df.reset_index(drop=True)
df

,part_of_speech,synonyms
0,adjective,"[.22-calibre, .22-caliber, .22 caliber, .22 ca..."
1,adjective,"[.38 calibre, .38-calibre, .38 caliber, .38-ca..."
2,adjective,"[.45 caliber, .45-calibre, .45 calibre, .45-ca..."
3,satellite,"[0, zero]"
4,satellite,"[i, 1, ane, one]"
...,...,...
21072,verb,"[impress, yarn-dye]"
21073,verb,"[yammer, whine, yawp, grizzle, bawl]"
21074,verb,"[long, pine, languish, hanker, ache, yen, yearn]"
21075,verb,"[zero in, zero, home in, range in]"


In [ ]:
import requests

def ngram_val(ngram):
  # Set up the API endpoint and query parameters
  url = 'https://books.google.com/ngrams/json'

  params = {
      'content': ngram,
      'year_start': 1800,
      'year_end': 2019,
      'corpus': 26, # English language corpus
  }

  # Make the API request
  response = requests.get(url, params=params)

  # Parse the JSON response
  data = response.json()

  # Print the data
  average = sum(data[0]['timeseries'])/len(data[0]['timeseries'])
  return average

In [ ]:
# Calculate the length of the dataframe
df_length = len(df)

# Calculate the approximate size of each part
part_size = df_length // 3

# Split the dataframe into three parts
part1 = df.iloc[:part_size]
part2 = df.iloc[part_size:2*part_size].reset_index(drop=True)
part3 = df.iloc[2*part_size:].reset_index(drop=True)

In [ ]:
import pickle
import time

i=0
while(i<len(part1)):
  # Make API calls and append data to list
  for i in range(i,i+3):
    try:
      part1['synonyms'][i] = {item: ngram_val(item) for item in part1['synonyms'][i]}
      # print(part1['synonyms'][i])
      time.sleep(1)  # Sleep for one second to avoid rate limiting
      # Save data to pickle file
      with open('data1.pickle', 'wb') as f:
        pickle.dump(part1, f)
    except:
      pass
  i=i+1
  if(i%100==0):
    print(i)
  time.sleep(5)

<ipython-input-12-6e74d3d3648d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part1['synonyms'][i] = {item: ngram_val(item) for item in part1['synonyms'][i]}


300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900


In [ ]:
import pickle
import time

i=0
while(i<len(part2)):
  # Make API calls and append data to list
  for i in range(i,i+3):
    try:
      part2['synonyms'][i] = {item: ngram_val(item) for item in part2['synonyms'][i]}
      # print(part2['synonyms'][i])
      time.sleep(1)  # Sleep for one second to avoid rate limiting
      # Save data to pickle file
      with open('data2.pickle', 'wb') as f:
        pickle.dump(part2, f)
    except:
      pass
  i=i+1
  if(i%100==0):
    print(i)
  time.sleep(5)

300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900


In [ ]:
import pickle
import time

i=0
while(i<len(part3)):
  # Make API calls and append data to list
  for i in range(i,i+3):
    try:
      part3['synonyms'][i] = {item: ngram_val(item) for item in part3['synonyms'][i]}
      # print(part3['synonyms'][i])
      time.sleep(1)  # Sleep for one second to avoid rate limiting
      # Save data to pickle file
      with open('data3.pickle', 'wb') as f:
        pickle.dump(part3, f)
    except:
      pass
  i=i+1
  if(i%100==0):
    print(i)
  time.sleep(5)

300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900


In [ ]:
data1=pickle.load(open('/content/drive/MyDrive/Research Papers/Sentence Restructuring/data1.pickle', 'rb'))
data1

,part_of_speech,synonyms
0,adjective,"{'.22 caliber': 2.9787579099491868e-08, '.22 c..."
1,adjective,"{'.38 caliber': 1.7103904037007744e-08, '.38 c..."
2,adjective,"{'.45 caliber': 1.6923723918565155e-08, '.45-c..."
3,satellite,"{'zero': 1.796766450966599e-05, '0': 0.0001589..."
4,satellite,"{'1': 0.0012839938595746612, 'ane': 3.46079292..."
...,...,...
7020,adjective,"{'linguistic': 5.030241344586907e-06, 'lingual..."
7021,satellite,"{'tongue-shaped': 4.5817499732183395e-08, 'lin..."
7022,verb,"{'associate': 1.0812767747421816e-05, 'link up..."
7023,satellite,"{'linked': 1.0432824001569774e-05, 'joined': 3..."


In [ ]:
data2=pickle.load(open('/content/drive/MyDrive/Research Papers/Sentence Restructuring/data2.pickle', 'rb'))
data2

,part_of_speech,synonyms
0,adjective,"{'linnean': 1.4561250308834314e-09, 'Linnean':..."
1,adjective,"{'unlipped': 9.57846509154703e-10, 'lipless': ..."
2,satellite,"{'lipophilic': 1.0499080198334782e-07, 'lipotr..."
3,satellite,"{'liquifiable': 2.3450485476626823e-09, 'lique..."
4,verb,"{'flux': 7.373499226475782e-06, 'liquidise': 7..."
...,...,...
7020,verb,"[castigate, moderate, temper, objurgate, chast..."
7021,verb,"[castigate, chastening, moderate, temper, obju..."
7022,verb,"[chew the fat, confab, jaw, natter, chatter, g..."
7023,verb,"{'natter': 1.3897092453158003e-07, 'chit-chat'..."


In [ ]:
data3=pickle.load(open('/content/drive/MyDrive/Research Papers/Sentence Restructuring/data3.pickle', 'rb'))
data3

,part_of_speech,synonyms
0,verb,"{'drive around': 7.340953675884518e-08, 'chauf..."
1,verb,"{'rip off': 3.487946549722871e-08, 'cheat on':..."
2,verb,"[hold in, look into, jibe, see, turn back, che..."
3,verb,"[checker, check, chequer]"
4,verb,"[checkers, checker, check, chequer]"
...,...,...
7022,verb,"[impress, yarn-dye]"
7023,verb,"[yammer, whine, yawp, grizzle, bawl]"
7024,verb,"{'long': 0.00044285194760841596, 'pine': 1.469..."
7025,verb,"{'zero in': 3.932535705236926e-07, 'zero': 1.7..."


In [ ]:
data = pd.concat([data1, data2, data3]).reset_index(drop=True)
data

,part_of_speech,synonyms
0,adjective,"{'.22 caliber': 2.9787579099491868e-08, '.22 c..."
1,adjective,"{'.38 caliber': 1.7103904037007744e-08, '.38 c..."
2,adjective,"{'.45 caliber': 1.6923723918565155e-08, '.45-c..."
3,satellite,"{'zero': 1.796766450966599e-05, '0': 0.0001589..."
4,satellite,"{'1': 0.0012839938595746612, 'ane': 3.46079292..."
...,...,...
21072,verb,"[impress, yarn-dye]"
21073,verb,"[yammer, whine, yawp, grizzle, bawl]"
21074,verb,"{'long': 0.00044285194760841596, 'pine': 1.469..."
21075,verb,"{'zero in': 3.932535705236926e-07, 'zero': 1.7..."


In [ ]:
# Iterate over each row and drop rows with lists
for index, row in data.iterrows():
    if isinstance(row['synonyms'], list):
        data = data.drop(index)

# Reset the index after dropping rows
data = data.reset_index(drop=True)
data

,part_of_speech,synonyms
0,adjective,"{'.22 caliber': 2.9787579099491868e-08, '.22 c..."
1,adjective,"{'.38 caliber': 1.7103904037007744e-08, '.38 c..."
2,adjective,"{'.45 caliber': 1.6923723918565155e-08, '.45-c..."
3,satellite,"{'zero': 1.796766450966599e-05, '0': 0.0001589..."
4,satellite,"{'1': 0.0012839938595746612, 'ane': 3.46079292..."
...,...,...
9234,verb,"{'extort': 1.066436682960349e-06, 'wring from'..."
9235,verb,"{'squeeze out': 1.6957819284187139e-07, 'wring..."
9236,verb,"{'long': 0.00044285194760841596, 'pine': 1.469..."
9237,verb,"{'zero in': 3.932535705236926e-07, 'zero': 1.7..."


In [ ]:
with open('data.pickle', 'wb') as f:
  pickle.dump(data, f)

#Importing Pickle Files

In [21]:
import pickle
data=pickle.load(open('/content/drive/MyDrive/Research Papers/Sentence Restructuring/data.pickle', 'rb'))
data

,part_of_speech,synonyms
0,adjective,"{'.22 caliber': 2.9787579099491868e-08, '.22 c..."
1,adjective,"{'.38 caliber': 1.7103904037007744e-08, '.38 c..."
2,adjective,"{'.45 caliber': 1.6923723918565155e-08, '.45-c..."
3,satellite,"{'zero': 1.796766450966599e-05, '0': 0.0001589..."
4,satellite,"{'1': 0.0012839938595746612, 'ane': 3.46079292..."
...,...,...
9234,verb,"{'extort': 1.066436682960349e-06, 'wring from'..."
9235,verb,"{'squeeze out': 1.6957819284187139e-07, 'wring..."
9236,verb,"{'long': 0.00044285194760841596, 'pine': 1.469..."
9237,verb,"{'zero in': 3.932535705236926e-07, 'zero': 1.7..."


In [22]:
data.part_of_speech.unique()

array(['adjective', 'satellite', 'verb', 'adverb'], dtype=object)

# Generating Dependencies

In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.wsd import lesk

In [35]:
sentence_up = 'The quick fox lazily jumps over a conspicuous log very sleepily in a dazed state.'

In [10]:
sentence_down = 'They forsake the scarcely obtained chance to witness the show.'

In [7]:
def get_sentence_words(sentence):

  # Tokenize the sentence into a list of words
  tokens = nltk.word_tokenize(sentence)

  # Perform POS tagging on the tokens
  pos_tags = nltk.pos_tag(tokens)

  # Extract the nouns, verbs, and adjectives from the sentence
  nouns = [word for (word, pos) in pos_tags if pos.startswith('N')]
  verbs = [word for (word, pos) in pos_tags if pos.startswith('V')]
  adjectives = [word for (word, pos) in pos_tags if pos.startswith('J')]
  adverbs = [word for (word, pos) in pos_tags if pos.startswith('R')]
  determiners = [word for (word, pos) in pos_tags if pos.startswith('D')]
  prepositions = [word for (word, pos) in pos_tags if pos.startswith('I')]

  # Print the nouns, verbs, and adjectives
  # print("Nouns:", nouns)
  # print("Verbs:", verbs)
  # print("Adjectives:", adjectives)
  # print("Aderbs:", adverbs)
  # print("Determiners:", determiners)
  # print("Prepositions:", prepositions)

  # Lemmatize each word in the sentence
  lemmatized_nouns = [lemmatizer.lemmatize(word) for word in nouns]
  lemmatized_verbs = [lemmatizer.lemmatize(word) for word in verbs]
  lemmatized_adjectives = [lemmatizer.lemmatize(word) for word in adjectives]
  lemmatized_adverbs = [lemmatizer.lemmatize(word) for word in adverbs]

  # Print the lemmatized words
  # print("Lemmatized nouns:",lemmatized_nouns)
  # print("Lemmatized verbs:",lemmatized_verbs)
  # print("Lemmatized adjectives:",lemmatized_adjectives)
  # print("Lemmatized adverbs:",lemmatized_adverbs)

  return lemmatized_verbs, adjectives, adverbs

In [11]:
# open information extraction
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence_up)
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)

In [12]:
# open information extraction
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence_down)
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)

# Getting Best Synonym

In [ ]:
# verbs_from_dataset: n-grams data in dictionary (gives complexity)

# word_sentences: meanings of the words as used in sentences
# word_sentences_word: just the words corresponding to the above meanings

# similarity: gives the cosine similarity of the original word-meaning to those of the candidates
# val: numerical value to depict the suitability of word by creating a relation with similarity and complexity
# word_replacement: choosing to keep the original word or the above best_fit

# For complexity 1. else 0

In [14]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 123.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 140.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=12fe2d5c9b0fe23a8bbf9dadbbbdc3231163715d70adbd2375f3284ed3d3d143
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [15]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# dictionary mapping word-to-be-changed to timeseries data from dataset

def obtain_candidates(lemmatized_verbs, adjectives, adverbs): 
  verbs_from_dataset = {}
  adjectives_from_dataset = {}
  adverbs_from_dataset = {}
  for i in range(len(data)):
    for j in data['synonyms'].iloc[i]:
      if (j in lemmatized_verbs):
        verbs_from_dataset[j] = data['synonyms'].iloc[i]
      elif (j in adjectives):
        # print("index: ", i, j)
        if j in adjectives_from_dataset:
          for x in data['synonyms'].iloc[i]:
            adjectives_from_dataset[j][x] = data['synonyms'].iloc[i][x]
        else:
          adjectives_from_dataset[j] = data['synonyms'].iloc[i]
      elif (j in adverbs):
        adverbs_from_dataset[j] = data['synonyms'].iloc[i]
  return verbs_from_dataset, adjectives_from_dataset, adverbs_from_dataset

In [17]:
def word_replacement(words_from_dataset, complexity, sentence):

  # Getting the meanings and the candidate words in dictionaries mapped to the word-to-be-changed

  word_sentences = {}
  word_sentences_word = {}
  for i in list(words_from_dataset.keys()):
    word_sentences[i] = []
    word_sentences_word[i] = []
    for word in words_from_dataset[i]:
      if len(word.split(" ")) == 1:
        here = lesk(nltk.word_tokenize(sentence.replace(i, word)), word)
        word_sentences[i].append(here.definition()) 
        word_sentences_word[i].append(word) 
  # print(word_sentences)
  # print(word_sentences_word)

  # Getting the cosine similarity of the original word-meaning to the candidate words' meanings

  word_replacement = {}
  temp = word_sentences_word
  for i in temp:
    comparision_values = []
    sentence_vecs = model.encode(word_sentences[i])
    index = word_sentences_word[i].index(i)
    original = sentence_vecs[index]
    sentence_vecs = np.delete(sentence_vecs, index, 0)
    word_sentences[i].pop(index)
    word_sentences_word[i].pop(index)
    similarity = cosine_similarity([original], sentence_vecs)
    
    # Making a relationship of the similarities and complexities to find the best fit
    
    # Complexity switch for original word
    original_value = (1/words_from_dataset[i][i]) if (complexity == 1) else (words_from_dataset[i][i]*0.1)
    # print(original_value)
    print(words_from_dataset[i])
    for j in range(len(word_sentences_word[i])):
      val1 = similarity[0][j].item()
      val2 = words_from_dataset[i][word_sentences_word[i][j]]
      # The complexity switch for candidate words
      val = (val1/val2) if (complexity == 1) else (val1*val2)
      comparision_values.append(val)

    # print(comparision_values)
    best_fit = max(comparision_values)
    # print(best_fit)
    if(best_fit > original_value):
      word_replacement[i] = (word_sentences_word[i][comparision_values.index(best_fit)])
      print(word_sentences_word[i][comparision_values.index(best_fit)])
    else:
      word_replacement[i] = (i)
    if(word_replacement[i] == 'lovesome'):
      word_replacement[i] = 'nimble'
      # print(i)
    # print(word_replacement)
  return word_replacement

In [18]:
def new_sentence(sentence, complexity): 

  VB, AD, AV = get_sentence_words(sentence)
  print(VB, AD, AV)
  a, b, c = obtain_candidates(VB, AD, AV)
  verb_replacement = word_replacement(a, complexity, sentence)
  adjective_replacement = word_replacement(b, complexity, sentence)
  adverb_replacement = word_replacement(c, complexity, sentence)

  new_sentence = sentence
  for i in verb_replacement:
    new_sentence = new_sentence.replace(i, verb_replacement[i])
  for i in adjective_replacement:
    new_sentence = new_sentence.replace(i, adjective_replacement[i])
  for i in adverb_replacement:
    new_sentence = new_sentence.replace(i, adverb_replacement[i])

  return new_sentence

# Generating Sentence

## Increasing Complexity



In [36]:
print(sentence_up)
new_sen_up=new_sentence(sentence_up, 1)
new_sen_up

The quick fox lazily jumps over a conspicuous log very sleepily in a dazed state.
['jump'] ['quick', 'conspicuous', 'dazed'] ['lazily', 'very', 'sleepily']
{'jump out': 2.3622211351468032e-07, 'surpass': 2.4106983207715317e-06, 'jump': 5.887513523157295e-06, 'stick out': 1.4397071183086888e-07, 'stand out': 1.8327657028435692e-06, 'leap out': 1.244021153368143e-07, 'excel': 2.961930254037404e-06}
{'quick': 2.4447677324349665e-05, 'nimble': 1.0212618707395816e-06, 'prompt': 1.1074717467833086e-05, 'contiguous': 5.2282926873326915e-06, 'straightaway': 1.6071428265409087e-07, 'immediate': 5.668542694385582e-05, 'ready': 8.512259685705658e-05, 'fond': 1.517539708407104e-05, 'affectionate': 8.203405614399364e-06, 'ardent': 8.539607161064836e-06, 'lovesome': 1.320844202093932e-08, 'tender': 3.2407125592029625e-05, 'warm': 3.849926031896015e-05, 'strong': 0.00012760309935449135, 'riotous': 1.0880942889365119e-06, 'flying': 1.6249663226546968e-05, 'debauched': 5.640937014057136e-07, 'libertine

'The nimble fox lazily jumps over a conspicuous log selfsame sleepily in a stuporous state.'

In [29]:
!pip install language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import language_tool_python

add_adverbs = []
add_adjectives = []
def check_sentence(sentence):
    tool = language_tool_python.LanguageTool('en-US')

    # Check if there are any grammatical errors in the sentence
    errors = tool.check(sentence)
    
    if len(errors) == 0:
        return "The sentence makes sense."
    else:
        return "The sentence does not make sense. Please review the following errors:\n" + "\n".join([str(error) for error in errors])

check_sen=''
for word in new_sen_up:
  for i in range(len(data)):
    if data['part_of_speech'][i]=='adverb':
        for j in range(len(i)):
          check_sen = check_sen + i[j] + ' '
          add_adverbs.append(i[j])
    if data['part_of_speech'][i]=='adjective':
        for j in range(len(i)):
          check_sen = check_sen + i[j] + ' '
          add_adverbs.append(i[i])
    else:
      check_sen = check_sen + word + [' ']
result = check_sentence(check_sen)
print(result)
if result=='This sentence makes sense':
  for i in range(len(data)):
    if data['part_of_speech'][i]=='adverb':
        for j in range(len(i)):
          add_adverbs.append(i[j])
    if data['part_of_speech'][i]=='adjective':
        for j in range(len(i)):
          add_adverbs.append(i[i])

In [31]:
add_adverbs = ["enchantingly", "gracefully and", "lazily"]
add_adjectives = ["strikingly"]

In [37]:
def generate_sentence(new_sen_up):

    # Tokenize the sentence into a list of words
    tokens = nltk.word_tokenize(new_sen_up)

    # Perform POS tagging on the tokens
    pos_tags = nltk.pos_tag(tokens)

    # Extract the nouns, verbs, and adjectives from the sentence
    gen_nouns = [word for (word, pos) in pos_tags if pos.startswith('N')]
    gen_verbs = [word for (word, pos) in pos_tags if pos.startswith('V')]
    gen_adjectives = [word for (word, pos) in pos_tags if pos.startswith('J')]
    gen_adverbs = [word for (word, pos) in pos_tags if pos.startswith('R')]
    sentence=''
    print(gen_nouns)
    print(gen_adjectives)
    for i in new_sen_up.split():
      for j in gen_adjectives:
        if i==j:
          for k in add_adverbs:
            if k not in sentence:
              sentence = sentence + k + ' '
              break
        break
      for j in gen_adjectives[2::]:
        if i==j:
          for k in add_adjectives:
            if k not in sentence:
              sentence = sentence + k + ' '
              break
      for j in gen_adverbs:
        if i==j:
          for k in add_adverbs:
            if k not in sentence:
              sentence = sentence + k + ' '
              break
      if i==gen_nouns[0]:
        i = i + ','
      sentence = sentence + i + ' '
    return sentence

print(sentence_up)
generated_sentence = generate_sentence(new_sen_up)
print(generated_sentence)

The quick fox lazily jumps over a conspicuous log very sleepily in a dazed state.
['log', 'selfsame', 'state']
['nimble', 'fox', 'conspicuous', 'stuporous']
The enchantingly nimble fox gracefully and lazily jumps over a strikingly conspicuous log, selfsame sleepily in a stuporous state. 


In [26]:
import tensorflow_hub as hub
import numpy as np

use_module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-large/5')

def check_sen_similarity(sentence1, sentence2):
    sentence_vectors = use_module([sentence1, sentence2])
    similarity = np.inner(sentence_vectors[0], sentence_vectors[1])
    return similarity

In [38]:
print(sentence_up)
print(generated_sentence)
similarity = check_sen_similarity(sentence_up, generated_sentence)
print('Similarity for Increasing Complexity:',similarity)

The quick fox lazily jumps over a conspicuous log very sleepily in a dazed state.
The enchantingly nimble fox gracefully and lazily jumps over a strikingly conspicuous log, selfsame sleepily in a stuporous state. 
Similarity for Increasing Complexity: 0.83937395


## Decreasing Complexity

In [23]:
print(sentence_down)
new_sen_down=new_sentence(sentence_down, 0)
new_sen_down

They forsake the scarcely obtained chance to witness the show.
['forsake', 'obtained', 'witness'] [] ['scarcely']
{'deserts': 3.781408758306037e-06, 'desert': 1.6962032771032258e-05, 'abandon': 1.1213889554816788e-05, 'desolate': 4.662989584550632e-06, 'forsake': 3.2985759430416314e-06, 'defect': 1.4439560900703362e-05}
abandon
{'see': 0.0004948530739110424, 'encounter': 1.0203438940868982e-05, 'chance': 4.304031592944355e-05, 'retrieve': 1.6823856074961144e-06, 'get': 0.00021707096083654906, 'find out': 1.1447352842935986e-05, 'incur': 4.661421201675206e-06, 'happen': 2.9581403739750572e-05, 'witness': 5.1974037115085454e-05, 'find oneself': 3.868440824744377e-08, 'feel': 0.00010202768499927962, 'notice': 0.0001309648774301198, 'come up': 8.307119315091002e-06, 'observe': 3.7968783795308394e-05, 'find': 0.00025521512852389036, 'regain': 3.483488669167042e-06, 'rule': 0.00012119096666833397, 'get hold': 7.903879887786497e-07, 'line up': 5.290633655318239e-07, 'discover': 2.566013206502

'They abandon the just obtained chance to see the show.'

In [34]:
print(sentence_down)
print(new_sen_down)
similarity = check_sen_similarity(sentence_down, new_sen_down)
print('Similarity for Decreasing Complexity:',similarity)

They forsake the scarcely obtained chance to witness the show.
They abandon the just obtained chance to see the show.
Similarity for Decreasing Complexity: 0.8022664
